In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek\


from itertools import combinations

In [2]:
def create_new_spark_context(appName):
    return SparkSession.builder.appName(appName)\
    .master("local[*]").getOrCreate()

spark_session = create_new_spark_context("Q4")
sc = spark_session.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 21:00:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType([
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True),
        StructField("ORIGINE_CAR_KEY",StringType(),True),
        StructField("FINAL_CAR_KEY",StringType(),True),
        StructField("CHECK_STATUS_KEY", IntegerType(), True),
        StructField("COMPANY_ID", StringType(), True),
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [6]:
df = spark_session.read.csv('Sample_Traffic.csv', header=True, schema=schema)
df.show(1)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



In [5]:
traffic_rdd = df.rdd.map(lambda x: ((x["FINAL_CAR_KEY"] , x["PASS_DAY_TIME"].date()) , x["DEVICE_CODE"]))\
                    .groupByKey()\
                    .map(lambda x: (x[0], sorted(list(set(list(x[1]))))))\
                    .filter(lambda x: len(x[1]) < 20)\
                    .map(lambda x: x[1])

paths_rdd = traffic_rdd.map(lambda x: x[1])

In [7]:
frequent_paths = {}

In [8]:
def prune_candidates(x, Ck, n):
    combs = list(combinations(x, n))
    return all(i in Ck for i in combs)


def apriori(traffic_rdd, support_threshold, n):    
    if n == 1:
        f_1 = traffic_rdd.flatMap(lambda x: x)\
                .map(lambda x: (x, 1))\
                .reduceByKey(lambda x, y: x + y)\
                .filter(lambda x: x[1] >= support_threshold)\
                .map(lambda x : ([x[0]], x[1]))\
                .collect()
        f_1 = {tuple(x[0]): x[1] for x in f_1}
        frequent_paths[1] = f_1
        return
    
    Cn_1 = apriori(traffic_rdd, support_threshold, n - 1)
    f_n = traffic_rdd.flatMap(lambda x: combinations(x, n))\
                .filter(lambda x: prune_candidates(x, Cn_1, n - 1))\
                .map(lambda x: (x, 1))\
                .reduceByKey(lambda x, y: x + y)\
                .filter(lambda x: x[1] >= support_threshold)\
                .collect()
    f_n = {tuple(x[0]): x[1] for x in f_n}
    frequent_paths[n] = f_n
    return f_n

In [9]:
support_threshold = 1000

In [10]:
f_5 = apriori(traffic_rdd, support_threshold, 5)

22/12/20 21:04:46 ERROR Executor: Exception in task 7.0 in stage 6.0 (TID 45)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/rdd.py", line 540, in func
    return f(iterator)
  File "/Users

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 6.0 failed 1 times, most recent failure: Lost task 4.0 in stage 6.0 (TID 42) (172.27.161.205 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/pyspark/rdd.py", line 

In [14]:
frequent_paths

{}

In [ ]:
traffic_rdd = traffic_rdd.zipWithIndex()


def check_4_tuples_son(x):
    res = []
    four_members_in_x = itertools.combinations(set(x[1]) , 4)
    for item in four_members_in_x:
        if(item in son_candidates):
            res.append((item , 1))
    return res


def check_son_tuples(x, n):
    res = []
    four_members_in_x = combinations(set(x[1]) , n)
    for item in four_members_in_x:
        if(item in son_candidates):
            res.append((item , 1))
    return res


def SON(traffic_rdd, support_threshold, n):
    
    traffic_rdd_1 = traffic_rdd.filter(lambda x : x[1]%3==0).map(lambda x : x[0])
    traffic_rdd_2 = traffic_rdd.filter(lambda x : x[1]%3==1).map(lambda x : x[0])
    traffic_rdd_3 = traffic_rdd.filter(lambda x : x[1]%3==2).map(lambda x : x[0])
    
    support_threshold = support_threshold / 3
    
    f_n_1 = apriori(traffic_rdd_1, support_threshold, n)
    f_n_2 = apriori(traffic_rdd_2, support_threshold, n)
    f_n_3 = apriori(traffic_rdd_3, support_threshold, n)
    
    
    
    
    
    

In [ ]:
f_n_1 = apriori(traffic_rdd_1, support_threshold, n)
f_n_2 = apriori(traffic_rdd_2, support_threshold, n)
f_n_3 = apriori(traffic_rdd_3, support_threshold, n)

In [ ]:
# combine f_n_1, f_n_2, f_n_3
f_n = {}
for key in f_n_1:
    f_n[key] = f_n_1[key]
for key in f_n_2:
    if key in f_n:
        f_n[key] += f_n_2[key]
    else:
        f_n[key] = f_n_2[key]
for key in f_n_3:
    if key in f_n:
        f_n[key] += f_n_3[key]
    else:
        f_n[key] = f_n_3[key]


In [ ]:
# 